In [14]:
from linearmodels import IV2SLS
import pandas as pd
import os
import numpy as np

In [5]:
top_down_path = os.getcwd()
os.chdir("../Data")
data_path = os.getcwd()
os.chdir(top_down_path)

In [46]:
# Read preprocessed
df = pd.read_csv(os.path.join(data_path, "preprocessed.csv"), index_col=0)
df.head()

,aggregated(no_dist_heat),dist_heat,slp_the [TWh],rlm_the [TWh],rlm_the_res,bdew [TWh],bdew_res,scaled_to_AGEBL_2020 [TWh_gas/day],gas_price [$/lhs],index,...,year,month,weekday,weekday_num,monday,friday,weekend,christmas_period,holiday,bridgeday
2021-10-01,0.738166,0.037952,0.701800,1.304783,1.075785,2.046847,1.817849,0.228998,93.62,2021-10-01,...,2021,10,Friday,4,0,1,0,0,0,0
2021-10-02,0.559080,0.028744,0.610800,1.077491,0.839662,1.802717,1.564888,0.237829,NaN,2021-10-02,...,2021,10,Saturday,5,0,0,1,0,0,0
2021-10-03,0.396701,0.020396,0.506066,1.074001,0.683518,1.707964,1.317480,0.390484,NaN,2021-10-03,...,2021,10,Sunday,6,0,0,1,0,1,0
2021-10-04,0.510587,0.026251,0.565016,1.415380,1.114668,1.932167,1.631455,0.300713,96.65,2021-10-04,...,2021,10,Monday,0,1,0,0,0,0,0
2021-10-05,0.618396,0.031794,0.638871,1.393310,1.053295,2.226157,1.886142,0.340015,116.02,2021-10-05,...,2021,10,Tuesday,1,0,0,0,0,0,0


# Estimations: X^= a + b*X

In [78]:
df_x = df

df_x = df_x.rename(columns = {
    "aggregated(no_dist_heat)" : "slp_w2h",
    "slp_the [TWh]" : "slp_the",
    "slp_the [TWh]" : "slp_the",
    "bdew [TWh]" : "bdew",
    "bdew_res" : "bdew_res",
    "rlm_the [TWh]" : "rlm_the",
    "rlm_the_res" : "rlm_the_res",
    "gas_price [$/lhs]" : "prices",
    "scaled_to_AGEBL_2020 [TWh_gas/day]" : "power"
})

df_x["the_sum"] = df_x["rlm_the"].iloc[:, 0] + df_x["slp_the"].iloc[:, 0]


In [79]:
df_x_no_holiday = df_x[(df_x.holiday==0) & (df_x.christmas_period==0) & (df_x.bridgeday==0)]

In [85]:
df_x_no_holiday.columns

Index(['slp_w2h', 'dist_heat', 'slp_the', 'rlm_the', 'rlm_the_res', 'bdew',
       'bdew_res', 'power', 'prices', 'index', 'time', 'year', 'month',
       'weekday', 'weekday_num', 'monday', 'friday', 'weekend',
       'christmas_period', 'holiday', 'bridgeday', 'slp_w2h', 'slp_the',
       'bdew', 'rlm_the', 'the', 'the_res', 'prices', 'power', 'the_sum'],
      dtype='object')

## bdew

In [122]:
formula_bdew = 'bdew_res ~ 1 + slp_w2h + C(weekend, month, friday)'

In [123]:
model = IV2SLS.from_formula(formula, data=df_x_no_holiday)
results = model.fit() 

In [124]:
results

OLS Estimation Summary                            
==============================================================================
Dep. Variable:             bdew[bdew]   R-squared:                      0.8156
Estimator:                        OLS   Adj. R-squared:                 0.8132
No. Observations:                 156   F-statistic:                    694.43
Date:                Tue, Apr 12 2022   P-value (F-stat)                0.0000
Time:                        10:56:56   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                                Parameter Estimates                                 
====================================================================================
                  Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------------
Intercept            1.1564     0.0816     14.178     0.0000      0.9966      1.3163
C(weekend)[T.1]     -0.2985     0.0546    -5.4711     0.0000     -0.4055     -0.1916
slp_w2h[slp_w2h]     1.5192     0.0577     26.314     0.0000      1.4060      1.6323
====================================================================================
OLSResults, id: 0x7fcd797c5700

In [ ]:
d_R2 = {"bdew":
     {"weekend":0.8156,
      "weekday":0.8156,
       "weekday":0.8156
     }
    
}

In [ ]:
Next steps:
RLM* = RLM_THE_res (- X * SLP)

bdew auf 2021 fitten mit a(SLP) 

plot estimate_bdew_2021 vs bdew_2021

Powersektor
1) nur residual und plot estimate_res + power vs bdew_res + power 
2) als Kontrollvariable: Ewartung: 25% 

model.fit ohne weihnacht, dann plot, dann model.predict für 2022

In [ ]:
formula = 'bdew ~ 1 + slp_w2h + C(weekend)'

formula = ('load') + ' ~ 1 + ' + ' + '.join(
    controls + ['C({})'.format(d) for d in time_dummies]
)

In [ ]:
pd.concat(
    [results.params, results.std_errors], axis=1
)#.to_csv('results.csv')

In [89]:
def extract(x):
    #x1 = x.first_stage.diagnostics
    return {
        'estimate': x.params['price'],
        'errors': x.std_errors['price']#,
        #'prsq': x1['partial.rsquared']['price'],
        #'fstat': x1['f.stat']['price']
    }